In [ ]:
import tensorflow as tf
import shutil
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('creditcard.csv')

In [ ]:
## data setting
# setting up testing and training sets
df_train, df_test = train_test_split(data, test_size=0.2, random_state=27)

print('df_train :', df_train.shape)
print('df_test :', df_test.shape)
print('df_train Class: \n', df_train.Class.value_counts())

X_test, y_test -> 최종 평가를 위한 데이터

문제를 풀기위한 데이터 셋 구성

In [ ]:
## Train_dataset Generate synthetic samples  (upsampling) --> 랜덤하게 포인트 뿌리고 knn을 이용하여 레이블 달아 데이터를 생성하는 방식
sm = SMOTE(random_state=27, ratio=1.0)
SMOTE_X_train, SMOTE_y_train = sm.fit_sample(df_train, df_train.Class)
df_train = pd.DataFrame(SMOTE_X_train, columns=list(df_train))
print(df_train.Class.value_counts())

In [ ]:
df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=27)
print('df_train :', df_train.shape)
print('df_valid :', df_valid.shape)
print('df_test :', df_test.shape)

In [ ]:
# feature_cols
# 일단 feature 형태 구분없이 모든 피처를 사용하는 것으로 배포
# 추후 feature engineering 후 재 배포 예정
FEATURE_NAMES = list(df_train)[:-1]
LABEL_NAME = 'Class'
feature_cols = [tf.feature_column.numeric_column(k) for k in list(df_train[FEATURE_NAMES])]

In [ ]:
# pandas_input_fn
def pd_input_fn(df,batch_size=128,num_epochs=5,shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
      x=df[FEATURE_NAMES],
      y=df[LABEL_NAME],
      batch_size = batch_size,
      num_epochs = num_epochs,
      shuffle = shuffle,
      queue_capacity = 1000,
      num_threads = 1
  )

----

샘플로 모델 돌리기 코드(train, eval 구분) -- test는 제외 --> 최종 평가에 쓰일 예정

In [ ]:
OUTDIR = "credit-prac"

config = tf.estimator.RunConfig(
    model_dir = OUTDIR,
    tf_random_seed = 1,
    save_checkpoints_steps = 100
)

model = tf.estimator.DNNClassifier(
  hidden_units = [10,10], 
  feature_columns = feature_cols, 
  config = config
)

In [ ]:
%%time
tf.logging.set_verbosity(tf.logging.INFO) 
shutil.rmtree(path = OUTDIR, ignore_errors = True)

model.train(input_fn = pd_input_fn(df=df_train,num_epochs=5), steps = 500)

In [ ]:
metrics = model.evaluate(input_fn=pd_input_fn(df=df_valid, batch_size=128, num_epochs=1, shuffle=False))